In [2]:
getwd()
if ( is.null(environment()$this_notebook_dir) ) {
    this_notebook_dir <- getwd()
    setwd(paste0(getwd(), '/..'))
}
getwd()

[1] "/mnt/hdd/develop/kaggle/house-prices/R"

[1] "/mnt/hdd/develop/kaggle/house-prices/R"

In [4]:
source('helpers.R')
source('validation_utils.R')

kaggle.house.loadLibraries()
data <- kaggle.house.loadData()

training <- data$train
testing <- data$test

c(dim(training), dim(testing))

[1] 1460   81 1459   80

In [6]:
training %>% purrr::map(~is.character(.)) %>% purrr::keep(~.) %>% names %>% sort

[1] "Alley"         "BldgType"      "BsmtCond"      "BsmtExposure" 
 [5] "BsmtFinType1"  "BsmtFinType2"  "BsmtQual"      "CentralAir"   
 [9] "Condition1"    "Condition2"    "Electrical"    "ExterCond"    
[13] "Exterior1st"   "Exterior2nd"   "ExterQual"     "Fence"        
[17] "FireplaceQu"   "Foundation"    "Functional"    "GarageCond"   
[21] "GarageFinish"  "GarageQual"    "GarageType"    "Heating"      
[25] "HeatingQC"     "HouseStyle"    "KitchenQual"   "LandContour"  
[29] "LandSlope"     "LotConfig"     "LotShape"      "MasVnrType"   
[33] "MiscFeature"   "MSZoning"      "Neighborhood"  "PavedDrive"   
[37] "PoolQC"        "RoofMatl"      "RoofStyle"     "SaleCondition"
[41] "SaleType"      "Street"        "Utilities"

In [7]:
training %>% group_by(Alley) %>% count

tranform.Alley <- function (df, remove=T) {
    df <- df %>% mutate(has_alley_access = ifelse(!is.na(Alley), 1, 0))
    if ( remove ) {
        df$Alley <- NULL
    }
    df
}

tranformed <- tranform.Alley(training, remove=F)
tranformed %>% group_by(has_alley_access, Alley) %>% count

Alley,n
Grvl,50
Pave,41
NA,1369


has_alley_access,Alley,n
0,NA,1369
1,Grvl,50
1,Pave,41


In [8]:
training %>% group_by(CentralAir) %>% count

tranform.CentralAir <- function (df, remove=T) {
    df <- df %>% mutate(has_central_air = ifelse(CentralAir == 'Y', 1, 0))
    if ( remove ) {
        df$CentralAir <- NULL
    }
    df
}

tranformed <- tranform.CentralAir(training, remove=F)
tranformed %>% group_by(has_central_air, CentralAir) %>% count

CentralAir,n
N,95
Y,1365


has_central_air,CentralAir,n
0,N,95
1,Y,1365


In [9]:
training %>% group_by(Heating) %>% count

tranform.Heating <- function (df, remove=T) {
    df <- df %>% mutate(heating_air_furnace = ifelse(Heating == 'GasA', 1, 0))
    if ( remove ) {
        df$Heating <- NULL
    }
    df
}

tranformed <- tranform.Heating(training, remove=F)
tranformed %>% group_by(heating_air_furnace, Heating) %>% count

Heating,n
Floor,1
GasA,1428
GasW,18
Grav,7
OthW,2
Wall,4


heating_air_furnace,Heating,n
0,Floor,1
0,GasW,18
0,Grav,7
0,OthW,2
0,Wall,4
1,GasA,1428


In [10]:
training %>% group_by(LandContour) %>% count

tranform.LandContour <- function (df, remove=T) {
    df <- df %>% mutate(is_land_level = ifelse(LandContour == 'Lvl', 1, 0))
    if ( remove ) {
        df$Heating <- NULL
    }
    df
}

tranformed <- tranform.LandContour(training, remove=F)
tranformed %>% group_by(is_land_level, LandContour) %>% count

LandContour,n
Bnk,63
HLS,50
Low,36
Lvl,1311


is_land_level,LandContour,n
0,Bnk,63
0,HLS,50
0,Low,36
1,Lvl,1311


In [11]:
training %>% group_by(LotShape) %>% count

tranform.LotShape <- function (df, remove=T) {
    df <- df %>% mutate(is_lotshape_regular = ifelse(LotShape == 'Reg', 1, 0))
    if ( remove ) {
        df$LotShape <- NULL
    }
    df
}

tranformed <- tranform.LotShape(training, remove=F)
tranformed %>% group_by(is_lotshape_regular, LotShape) %>% count

LotShape,n
IR1,484
IR2,41
IR3,10
Reg,925


is_lotshape_regular,LotShape,n
0,IR1,484
0,IR2,41
0,IR3,10
1,Reg,925


In [12]:
training %>% group_by(MiscFeature) %>% count

tranform.MiscFeature <- function (df, remove=T) {
    df <- df %>% mutate(has_misc_feature = ifelse(!is.na(MiscFeature), 1, 0))
    if ( remove ) {
        df$MiscFeature <- NULL
    }
    df
}

tranformed <- tranform.MiscFeature(training, remove=F)
tranformed %>% group_by(has_misc_feature, MiscFeature) %>% count

MiscFeature,n
Gar2,2
Othr,2
Shed,49
TenC,1
NA,1406


has_misc_feature,MiscFeature,n
0,NA,1406
1,Gar2,2
1,Othr,2
1,Shed,49
1,TenC,1


In [14]:
training %>% group_by(PavedDrive) %>% count

tranform.PavedDrive <- function (df, remove=T) {
    df <- df %>% mutate(has_paved_drive = ifelse(PavedDrive == 'Y', 1, 0))
    if ( remove ) {
        df$PavedDrive <- NULL
    }
    df
}

tranformed <- tranform.PavedDrive(training, remove=F)
tranformed %>% group_by(has_paved_drive, PavedDrive) %>% count

PavedDrive,n
N,90
P,30
Y,1340


has_paved_drive,PavedDrive,n
0,N,90
0,P,30
1,Y,1340


In [17]:
training %>% group_by(PoolQC) %>% count

tranform.PoolQC <- function (df, remove=T) {
    df <- df %>% mutate(has_pool = ifelse(!is.na(PoolQC), 1, 0))
    if ( remove ) {
        df$PoolQC <- NULL
    }
    df
}

tranformed <- tranform.PoolQC(training, remove=F)
tranformed %>% group_by(has_pool, PoolQC) %>% count

PoolQC,n
Ex,2
Fa,2
Gd,3
NA,1453


has_pool,PoolQC,n
0,NA,1453
1,Ex,2
1,Fa,2
1,Gd,3


In [21]:
training %>% group_by(RoofMatl) %>% count

tranform.RoofMatl <- function (df, remove=T) {
    df <- df %>% mutate(has_standard_roof_material = ifelse(RoofMatl == 'CompShg', 1, 0))
    if ( remove ) {
        df$RoofMatl <- NULL
    }
    df
}

tranformed <- tranform.RoofMatl(training, remove=F)
tranformed %>% group_by(has_standard_roof_material, RoofMatl) %>% count

RoofMatl,n
ClyTile,1
CompShg,1434
Membran,1
Metal,1
Roll,1
Tar&Grv,11
WdShake,5
WdShngl,6


has_standard_roof_material,RoofMatl,n
0,ClyTile,1
0,Membran,1
0,Metal,1
0,Roll,1
0,Tar&Grv,11
0,WdShake,5
0,WdShngl,6
1,CompShg,1434


In [26]:
training %>% group_by(Street) %>% count

tranform.Street <- function (df, remove=T) {
    df <- df %>% mutate(is_street_paved = ifelse(Street == 'Pave', 1, 0))
    if ( remove ) {
        df$Street <- NULL
    }
    df
}

tranformed <- tranform.Street(training, remove=F)
tranformed %>% group_by(is_street_paved, Street) %>% count

Street,n
Grvl,6
Pave,1454


is_street_paved,Street,n
0,Grvl,6
1,Pave,1454


In [27]:
training %>% purrr::map(~is.character(.)) %>% purrr::keep(~.) %>% names %>% sort

[1] "Alley"         "BldgType"      "BsmtCond"      "BsmtExposure" 
 [5] "BsmtFinType1"  "BsmtFinType2"  "BsmtQual"      "CentralAir"   
 [9] "Condition1"    "Condition2"    "Electrical"    "ExterCond"    
[13] "Exterior1st"   "Exterior2nd"   "ExterQual"     "Fence"        
[17] "FireplaceQu"   "Foundation"    "Functional"    "GarageCond"   
[21] "GarageFinish"  "GarageQual"    "GarageType"    "Heating"      
[25] "HeatingQC"     "HouseStyle"    "KitchenQual"   "LandContour"  
[29] "LandSlope"     "LotConfig"     "LotShape"      "MasVnrType"   
[33] "MiscFeature"   "MSZoning"      "Neighborhood"  "PavedDrive"   
[37] "PoolQC"        "RoofMatl"      "RoofStyle"     "SaleCondition"
[41] "SaleType"      "Street"        "Utilities"